# 2. PyTorch dataset and data loader

In the previous notebook, we have learned how to investigate our dataset. Next up, we'll want to create a PyTorch `Dataset` and a PyTorch `DataLoader`. These will help us get our data ready to be passed through a neural network.

## 2.1 Creating a PyTorch dataset

A [PyTorch `Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) is an abstract representation of a dataset. You can do two things with a PyTorch `Dataset`: get the *item* at a certain **index**, and get the **length** of the dataset (i.e., the number of *items*). That's all. What an *item* is, is entirely up to you, but in the case of classification, it is typically a tuple of an image and a class label.

### 2.1.1 Built-in datasets

Torchvision already includes [many datasets](https://pytorch.org/vision/stable/datasets.html) to play around with. Let's take a look at the [MNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST) dataset.

In [2]:
from torchvision.datasets import MNIST

mnist_ds = MNIST('../data/', train=True, download=True)

100%|███████████████████████████| 9912422/9912422 [00:00<00:00, 58442430.44it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|█████████████████████████████████| 28881/28881 [00:00<00:00, 118206.31it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|███████████████████████████| 1648877/1648877 [00:00<00:00, 36052188.90it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|█████████████████████████████████| 4542/4542 [00:00<00:00, 31230375.03it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



We can ask an item at a certain index:

In [5]:
mnist_ds[0]

(<PIL.Image.Image image mode=L size=28x28>, 5)

This gives us two things: an image and an integer label.

In [6]:
img, label = mnist_ds[0]

We can also ask the length of the dataset.

In [8]:
img


In [9]:
label

5

In [12]:
len(mnist_ds)

60000

### 2.1.2 A first attempt to a custom dataset

To implement your own dataset, you need create a class that inherits from [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) and implements two methods: `__getitem__()` and `__len__()`.

For our Pokémon use case:

- `__getitem__()` should return the image at the given index, along with the name of that Pokémon;
- `__len__()` should return the number of images.

In the previous notebook, we saw how we can represent our dataset as a Pandas `DataFrame`. We can use this representation to implement `__getitem__()` and `__len__()` without much new code:

In [7]:
from torch.utils.data import Dataset
from pathlib import Path 
import pandas as pd
from PIL import Image 

data_path = Path('../data/PokemonGen1')

# list comprehension way
df = pd.DataFrame ([
    {
    'image' : p,
    'label' : p.parent.name
    }
    for p in data_path.glob('data/*/*')
])

In [8]:
df.iloc[300] # is een rij

image    ../data/PokemonGen1/data/Cubone/5a87cbf8f94cdd...
label                                               Cubone
Name: 300, dtype: object

In [29]:
class MammalsDataset(Dataset):
    def __init__(self, datapath, transforms=None):
        data_path = Path(datapath)
        self.df = pd.DataFrame ([
            {
            'image' : p,
            'label' : p.parent.name
            }
            for p in data_path.glob('data/*/*')
        ])
        self.transforms = transforms
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        img = Image.open(img_path)
        if (self.transforms is not None):
            img = self.transforms(img)
        return img, label
        
    def __len__(self):
        return len(self.df)

We can play around with this dataset, just like with the MNIST dataset.

In [30]:
ds = MammalsDataset('../data/mammals')
ds[1]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256>, 'porcupine')

While this is already a perfectly valid PyTorch `Dataset`, there are two issues:

1. Our labels are strings;
2. We don't make a distinction between train, validation and test data.

> 🤔 **Why is it a problem that our labels are strings?**
>
> When training a network, we typically iterate over *batches* of data. These batches are represented as PyTorch tensors. And it is not possible (and not necessary, really) to create a tensor of strings.

We'll solve both issues in the following sections.

### 2.1.3 Replacing string labels with integer labels

First, we'll replace our string labels by integer labels. For this, we'll create a dictionary that maps each string label to a unique integer.

In [19]:
data_path = Path('../data/mammals')
df = pd.DataFrame([
    {
        'image': str(img_path),
        'label': img_path.parent.name
    }
    for img_path in data_path.glob('data/*/*')
])

To get a list of unique labels, we can call [`unique()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) on the `label` column:

In [20]:
df['label'].unique

<bound method Series.unique of 0            porcupine
1            porcupine
2            porcupine
3            porcupine
4            porcupine
             ...      
13746    water_buffalo
13747    water_buffalo
13748    water_buffalo
13749    water_buffalo
13750    water_buffalo
Name: label, Length: 13751, dtype: object>

To ensure a consistent mapping across different systems, we can sort these labels with [the built-in Python function `sorted()`](https://docs.python.org/3/library/functions.html#sorted):

In [21]:
sorted(df['label'].unique())

['african_elephant',
 'alpaca',
 'american_bison',
 'anteater',
 'arctic_fox',
 'armadillo',
 'baboon',
 'badger',
 'blue_whale',
 'brown_bear',
 'camel',
 'dolphin',
 'giraffe',
 'groundhog',
 'highland_cattle',
 'horse',
 'jackal',
 'kangaroo',
 'koala',
 'manatee',
 'mongoose',
 'mountain_goat',
 'opossum',
 'orangutan',
 'otter',
 'polar_bear',
 'porcupine',
 'red_panda',
 'rhinoceros',
 'sea_lion',
 'seal',
 'snow_leopard',
 'squirrel',
 'sugar_glider',
 'tapir',
 'vampire_bat',
 'vicuna',
 'walrus',
 'warthog',
 'water_buffalo',
 'weasel',
 'wildebeest',
 'wombat',
 'yak',
 'zebra']

Now, with [the built-in Python function `enumerate()`](https://docs.python.org/3/library/functions.html#enumerate), we can get an iterable that yields another integer value for each of our labels.

In [25]:
for i, label in enumerate(sorted(df['label'].unique())):
    print(i, label)

0 african_elephant
1 alpaca
2 american_bison
3 anteater
4 arctic_fox
5 armadillo
6 baboon
7 badger
8 blue_whale
9 brown_bear
10 camel
11 dolphin
12 giraffe
13 groundhog
14 highland_cattle
15 horse
16 jackal
17 kangaroo
18 koala
19 manatee
20 mongoose
21 mountain_goat
22 opossum
23 orangutan
24 otter
25 polar_bear
26 porcupine
27 red_panda
28 rhinoceros
29 sea_lion
30 seal
31 snow_leopard
32 squirrel
33 sugar_glider
34 tapir
35 vampire_bat
36 vicuna
37 walrus
38 warthog
39 water_buffalo
40 weasel
41 wildebeest
42 wombat
43 yak
44 zebra


So, we can create our label-to-integer dictionary with the following `for` loop:

In [27]:
label_to_int = {}
for i, label in enumerate(sorted(df['label'].unique())):
    label_to_int[label] = i
label_to_int

{'african_elephant': 0,
 'alpaca': 1,
 'american_bison': 2,
 'anteater': 3,
 'arctic_fox': 4,
 'armadillo': 5,
 'baboon': 6,
 'badger': 7,
 'blue_whale': 8,
 'brown_bear': 9,
 'camel': 10,
 'dolphin': 11,
 'giraffe': 12,
 'groundhog': 13,
 'highland_cattle': 14,
 'horse': 15,
 'jackal': 16,
 'kangaroo': 17,
 'koala': 18,
 'manatee': 19,
 'mongoose': 20,
 'mountain_goat': 21,
 'opossum': 22,
 'orangutan': 23,
 'otter': 24,
 'polar_bear': 25,
 'porcupine': 26,
 'red_panda': 27,
 'rhinoceros': 28,
 'sea_lion': 29,
 'seal': 30,
 'snow_leopard': 31,
 'squirrel': 32,
 'sugar_glider': 33,
 'tapir': 34,
 'vampire_bat': 35,
 'vicuna': 36,
 'walrus': 37,
 'warthog': 38,
 'water_buffalo': 39,
 'weasel': 40,
 'wildebeest': 41,
 'wombat': 42,
 'yak': 43,
 'zebra': 44}

Or, we can use a [dictionary comprehension](https://www.geeksforgeeks.org/python-dictionary-comprehension/):

In [28]:
label_to_int = {label : i for i, label in enumerate(sorted(df['label'].unique()))}
label_to_int

{'african_elephant': 0,
 'alpaca': 1,
 'american_bison': 2,
 'anteater': 3,
 'arctic_fox': 4,
 'armadillo': 5,
 'baboon': 6,
 'badger': 7,
 'blue_whale': 8,
 'brown_bear': 9,
 'camel': 10,
 'dolphin': 11,
 'giraffe': 12,
 'groundhog': 13,
 'highland_cattle': 14,
 'horse': 15,
 'jackal': 16,
 'kangaroo': 17,
 'koala': 18,
 'manatee': 19,
 'mongoose': 20,
 'mountain_goat': 21,
 'opossum': 22,
 'orangutan': 23,
 'otter': 24,
 'polar_bear': 25,
 'porcupine': 26,
 'red_panda': 27,
 'rhinoceros': 28,
 'sea_lion': 29,
 'seal': 30,
 'snow_leopard': 31,
 'squirrel': 32,
 'sugar_glider': 33,
 'tapir': 34,
 'vampire_bat': 35,
 'vicuna': 36,
 'walrus': 37,
 'warthog': 38,
 'water_buffalo': 39,
 'weasel': 40,
 'wildebeest': 41,
 'wombat': 42,
 'yak': 43,
 'zebra': 44}

We can plug this into our mamalDataset:

In [31]:
class MammalsDataset(Dataset):
    def __init__(self, datapath, transforms=None):
        data_path = Path(datapath)
        df = pd.DataFrame ([
            {
            'image' : p,
            'label' : p.parent.name
            }
            for p in data_path.glob('data/*/*')
        ])

        # mappen van label naar integer
        label_to_int = {label : i for i, label in enumerate(sorted(df['label'].unique()))}
        
        # store attributes
        self.data_path = data_path
        self.label_to_int = label_to_int
        self.transforms = transforms
        self.df = df
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        int_label = self.label_to_int[label]
        img = Image.open(img_path)
        if (self.transforms is not None):
            img = self.transforms(img)
        return (img, label)
        
    def __len__(self):
        return len(self.df)

Let's try out our new dataset.

In [45]:
ds = MammalsDataset('../data/mammals/')
ds[10]

TypeError: MammalsDataset.__init__() missing 1 required positional argument: 'subset'

### 2.1.4 Splitting the data into a train, validation and test split

Training a neural network and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would achieve a perfect score but would fail to predict anything useful on data that was not used during training. This situation is called **overfitting**. To get a better sense of model performance on *unseen* data, it is customary to randomly split the dataset into three disjoint subsets.

1. **Training set**: You train the model with the training set.
2. **Validation set**: Every X training iterations, you evaluate the model performance on the validation set. It is common to only keep the model that obtained the best validation score during training. When training multiple models with different configurations (or *hyperparameters*), you should use the evaluation on the validation set to decide which model to keep.
3. **Test set**: *Once you have decided a model based on validation perfomance*, you evaluate on the test set to get an estimate of the model's ability to generalize to unseen data. In many machine learning competitions, to avoid cheating, the test set is either unlabeled or unavailable to the participants.

As you see, the validation set is used to compare different model configurations, or *hyperparameters*. This could simply be the number of iterations used to train the model, but this also includes the choice of data augmentations, optimizer, learning rate, model architecture (number of layers, number of neurons in a layer, kind of layer,...) etc.

To split our dataset into these subsets, we can use [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split).

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
df_train, df_val = train_test_split(df_trainval, train_size=0.8, random_state=42)

We can update our `mammals` so that, depending on a given constructor argument, the correct subset is used.

In [54]:
class MammalsDataset(Dataset):
    def __init__(self, datapath, subset, transforms=None):
        data_path = Path(datapath)
        df = pd.DataFrame ([
            {
            'image' : p,
            'label' : p.parent.name
            }
            for p in data_path.glob('data/*/*')
        ])

        # mappen van label naar integer
        label_to_int = {label : i for i, label in enumerate(sorted(df['label'].unique()))}

        # subsets data
        df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
        df_train, df_val = train_test_split(df_trainval, train_size=0.8, random_state=42)
            
        # store attributes
        self.data_path = data_path
        self.label_to_int = label_to_int
        self.transforms = transforms
        self.df = df
        self.subset = subset

        if subset == 'train':
            self.df = df_train.reset_index()
        elif subset == 'val':
            self.df = df_val.reset_index()
        elif subset == 'test':
            self.df = df_test.reset_index()
        else:
            raise ValueError(f'Unknown subset {subset}')
    
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        int_label = self.label_to_int[label]
        img = Image.open(img_path).convert('RGB')
        if (self.transforms is not None):
            img = self.transforms(img)
        return (img, label)
        
    def __len__(self):
        return len(self.df)

Let's try it out...

In [55]:
ds_train = MammalsDataset(datapath='../data/mammals', subset='train')
ds_train

### 2.1.5 Cross-validation

Choosing the model that achieved the best performance on the validation set (instead of the training set) avoids selecting a model that has overfit. However, when comparing many models based on validation score, we might select a model that only worked best for the particular random choice of training and validation data. To make a better-informed decision, we can use **cross-validation** techniques.

A basic cross-validation technique is **$k$-fold cross-validation**. Here, the data set is first split up into only two subsets: a *training* set and a *test* set. The training set is partitioned into **$k$ equally-sized folds**. When training a model, we choose one fold for *validation* and the other $k - 1$ folds for *training*. By training a certain model configuration with all $k$ different choices for training and validation and averaging the validation scores, we can effectively select the best model configuration in a more reliable and robust manner. The figure below illustrates the process of $k$-fold cross-validation ([source](https://scikit-learn.org/stable/modules/cross_validation.html)).

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" style="max-width: 500px; margin: auto; padding: 2em;"/>

To add support for $k$-fold cross validation to our `mammals`, we'll first use `train_test_split()` to split our dataset into train+val and test set. Then, with [`numpy.array_split()`](https://numpy.org/doc/stable/reference/generated/numpy.array_split.html), we can split the train+val set into $k$ folds.

In [56]:
import numpy as np

In [67]:
# geef aantal k-fold
k = 5

# split de dataset in train + validatie
df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)

# split de training + validatie in k fold
folds = np.array_split(df_trainval, k)

In [73]:
folds[0]

,image,label
5263,../data/mammals/data/american_bison/american_b...,american_bison
4344,../data/mammals/data/sugar_glider/sugar_glider...,sugar_glider
3526,../data/mammals/data/tapir/tapir-0282.jpg,tapir
3519,../data/mammals/data/tapir/tapir-0130.jpg,tapir
9056,../data/mammals/data/wildebeest/wildebeest-025...,wildebeest
...,...,...
4718,../data/mammals/data/sea_lion/sea_lion-0265.jpg,sea_lion
1888,../data/mammals/data/snow_leopard/snow_leopard...,snow_leopard
711,../data/mammals/data/opossum/opossum-0290.jpg,opossum
12060,../data/mammals/data/brown_bear/brown_bear-019...,brown_bear


Now, we can choose which fold to use for validation. The other folds should be concatenated to create the training set.

In [75]:
fold_val = 3 # val fold kiezen 
df_val = fold[fold_val] # de array met validatie set

train_folds = []

for idx, df_fold in enumerate(folds):
    if idx == fold_val:
        continue
    else:
        train_folds.append(df_fold)

print(df_train)
df_train = pd.concat(train_folds)

                                                   image             label
6864   ../data/mammals/data/vampire_bat/vampire_bat-0...       vampire_bat
489    ../data/mammals/data/rhinoceros/rhinoceros-011...        rhinoceros
8685   ../data/mammals/data/highland_cattle/highland_...   highland_cattle
690        ../data/mammals/data/opossum/opossum-0059.jpg           opossum
602        ../data/mammals/data/opossum/opossum-0252.jpg           opossum
...                                                  ...               ...
13709  ../data/mammals/data/water_buffalo/water_buffa...     water_buffalo
6052   ../data/mammals/data/african_elephant/african_...  african_elephant
3178         ../data/mammals/data/vicuna/vicuna-0092.jpg            vicuna
3192         ../data/mammals/data/vicuna/vicuna-0102.jpg            vicuna
9499           ../data/mammals/data/otter/otter-0048.jpg             otter

[2200 rows x 2 columns]


Plugging this into our `mammals`:

In [76]:
class MammalsDataset(Dataset):
    def __init__(self, datapath, subset, k=5, val_fold=0, transforms=None):
        data_path = Path(datapath)
        df = pd.DataFrame ([
            {
            'image' : p,
            'label' : p.parent.name
            }
            for p in data_path.glob('data/*/*')
        ])

        # mappen van label naar integer
        label_to_int = {label : i for i, label in enumerate(sorted(df['label'].unique()))}

        # subsets of data
        df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
        df_train, df_val = train_test_split(df_trainval, train_size=0.8, random_state=42)

         # Split into train, test, val
        df_trainval, df_test = train_test_split(df, train_size=0.8, random_state=42)
        folds = np.array_split(df_trainval, k)
        df_val = folds[val_fold]
        train_folds = [fold for i, fold in enumerate(folds)
                       if i != val_fold]
        df_train = pd.concat(train_folds)
        
        # store attributes
        self.data_path = data_path
        self.label_to_int = label_to_int
        self.transforms = transforms

        if subset == 'train':
            self.df = df_train.reset_index()
        elif subset == 'val':
            self.df = df_val.reset_index()
        elif subset == 'test':
            self.df = df_test.reset_index()
        else:
            raise ValueError(f'Unknown subset {subset}')
    
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image']
        label = row['label']
        int_label = self.label_to_int[label]
        img = Image.open(img_path).convert('RGB')
        if (self.transforms is not None):
            img = self.transforms(img)
        return (img, label)
        
    def __len__(self):
        return len(self.df)

Now we can create a training set, validation set and test set:

In [80]:
import torch
from torchvision.transforms import v2

# Create transforms
train_tfms = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(30),
    v2.RandomResizedCrop(224, antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# mag geen random zijn bij validation pipeline (altijd zelfde input)
val_tfms = v2.Compose([
    v2.ToImage(),
    v2.Resize(224, antialias=True),
    v2.CenterCrop(224),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
ds_train = MammalsDataset('../data/mammals', 'train', transforms=train_tfms)
ds_val = MammalsDataset('../data/mammals', 'val', transforms=val_tfms)
ds_test = MammalsDataset('../data/mammals', 'test', transforms=val_tfms)

/home/student/labo_bld_interp/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/student/labo_bld_interp/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/student/labo_bld_interp/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


> ⚠️ **Don't use random transforms for validation or test set!**
>
> To avoid making the validation and test evaluations irreproducible, you should not use randomness in the transforms that you'll use for the validation and test set.

Hooray! 🙌 We now have a full-fledged PyTorch `Dataset` with support for $k$-fold cross-validation! 🎉

## 2.2 PyTorch data loader

A PyTorch `Dataset` allows us to easily get images (and labels) from our dataset. When training a neural network, however, we can save training time by training with **batches** of data, instead of passing the images through the network one-by-one.

Batching your data is the task of the [PyTorch `DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader). Given a PyTorch `Dataset` that returns a tuple with an image tensor and an integer label, it is very simple to create a `DataLoader`:

In [81]:
from torch.utils.data import DataLoader

# Create trainin data loader
dl_train = DataLoader(ds_train, batch_size=10)

A `DataLoader` is an iterable, where each iteration item is a batch of data samples. As with any iterable in Python, you can iterate over a `DataLoader` using a `for` loop. Let's inspect what's inside an iteration item.

In [82]:
for x in dl_train:
    break
x

[tensor([[[[-0.5596, -0.5596, -0.5596,  ..., -0.6623, -0.6623, -0.6623],
           [-0.5596, -0.5596, -0.5596,  ..., -0.6452, -0.6623, -0.6452],
           [-0.5596, -0.5596, -0.5596,  ..., -0.6452, -0.6452, -0.6452],
           ...,
           [ 0.9988,  1.0502,  1.0844,  ...,  0.7933,  0.9132,  1.1358],
           [ 1.2043,  1.0502,  0.8789,  ...,  1.0673,  0.8104,  0.7591],
           [ 1.2728,  1.2043,  1.1187,  ...,  1.0159,  1.0159,  1.2214]],
 
          [[ 0.5028,  0.5028,  0.5028,  ...,  0.4503,  0.4503,  0.4503],
           [ 0.5028,  0.5028,  0.5028,  ...,  0.4678,  0.4503,  0.4678],
           [ 0.5028,  0.5028,  0.5028,  ...,  0.4678,  0.4678,  0.4678],
           ...,
           [ 0.9405,  0.9930,  1.0280,  ...,  0.7129,  0.8354,  1.0630],
           [ 1.1506,  0.9930,  0.8179,  ...,  0.9930,  0.7304,  0.6779],
           [ 1.2206,  1.1506,  1.0630,  ...,  0.9580,  0.9580,  1.1681]],
 
          [[ 2.0125,  2.0125,  2.0125,  ...,  1.8731,  1.8731,  1.8731],
           [ 

Our `DataLoader` composes the first batch with the samples at index $0,\ldots, 9$, the second batch $10,\ldots,19$ and so on. To compose the batches with random samples, you can pass `shuffle=True` to the `DataLoader` constructor.

> ⚠️ **Only shuffle the training set!**
>
> It is considered bad practice to shuffle any of your evaluation sets, as this might make your evaluations irreproducible.

In [85]:
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)

When taking a look at the implementation of `__getitem__()` in our `PokemonDataset`, you'll see that it involves loading an image from disk with `read_image()` and applying image transforms. Both of these steps may block the computing process for a while. If we'd sequentially run `__getitem__()` for the 10 indices in our batch, it might take some time before the batch is created. Setting the argument `num_workers` as a positive integer will turn on **multi-process data loading** with the specified number of loader worker processes.

In [86]:
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True, num_workers=10)

In summary, the following code creates our train, validation and test datasets and data loaders.

In [87]:
# Create data loaders
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=10, shuffle=False)
dl_test = DataLoader(ds_test, batch_size=10, shuffle=False)

And with this, we have covered everything on the data side! 💪 Our data is now ready to be passed into a neural network!